In [1]:
import pandas as pd
import numpy as np

#Sklearn for model fitting and scroing
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [2]:
#Student name.
name = "Adewale Adeniji"

#EDA control. 
show_eda = False

In [3]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
20,1,0.638,0.094,0.697,0.910,0.391,0.028,0.567,0.535,0.067,...,0.173,0.062,0.949,0.063,0.085,0.870,0.149,0.845,0.848,0.502
35,1,0.842,0.339,0.893,0.036,0.378,0.477,0.485,0.485,0.602,...,0.457,0.930,0.684,0.340,0.308,0.536,0.638,0.414,0.530,0.785
124,0,0.244,0.030,0.627,0.907,0.047,0.127,0.073,0.615,0.032,...,0.730,0.364,0.125,0.291,0.138,0.494,0.921,0.595,0.399,0.445
39,1,0.945,0.604,0.381,0.724,0.863,0.378,0.724,0.950,0.663,...,0.951,0.568,0.413,0.225,0.434,0.810,0.866,0.583,0.462,0.671
25,1,0.043,0.650,0.220,0.274,0.010,0.543,0.409,0.353,0.078,...,0.167,0.597,0.295,0.670,0.828,0.444,0.512,0.002,0.136,0.857


### Starting

The first stage will be to look at the data in an initial exploratory data analysis. We know that our data is made up of numerical variables and is in a standardized form. The records are complete and there are non null values, so the data is in a clean state.

In [13]:
import ml_utils
df_eda = ml_utils.edaDF(df, "target")
num_var = list(df.columns)
df_eda.setNum(num_var)

In [14]:
df_eda.fullEDA()


In [6]:
df.shape

(250, 201)

### Model fitting

As mentioned above, our data is in a clean state and as such, we will proceed to model fitting. 
The model that is used is the DecisionTreeClassifier.
Conidering that the data set is small-sized (250 rows of data), we will use a 70/30 percent split for the train/test data.

In [15]:
y = np.array(df["target"]).reshape(-1,1)
X = np.array(df.drop(columns={"target"}))

#train size = 70%
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=.7)


Model 1: Here, the DecisionTreeClassifier is used with default hyperparameters 

In [16]:
pipeline_model1 = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(random_state=0))
    ]

model1 = Pipeline(pipeline_model1)
model1 = model1.fit(X_train, y_train)
print("Training Accuracy:", model1.score(X_train, y_train))
print("Testing Accuracy:", model1.score(X_test, y_test))

Training Accuracy: 1.0
Testing Accuracy: 0.6266666666666667


Model 2: The DecisionTreeClassifier is used with the criterion set to "entropy". 
We noticed that there is not so much difference in the testing accuracy 

In [17]:

pipeline_model2 = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(random_state=0, criterion="entropy"))
    ]
model2 = Pipeline(pipeline_model2)
model2 = model2.fit(X_train, y_train)
print("Training Accuracy:", model2.score(X_train, y_train))
print("Testing Accuracy:", model2.score(X_test, y_test))

Training Accuracy: 1.0
Testing Accuracy: 0.6133333333333333


Model 3: The DecisionTreeClassifier is used with the criterion set default and max_depth set to 5. 
There was improvement in the testing accuracy while the traininig accuracy dropped slightly. This was considered as the best model because further tuning seems not to yield better results.

In [19]:
#Build pipeline
pipeline_best = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(random_state=0, max_depth=5))
    ]

best = Pipeline(pipeline_best)
best = best.fit(X_train, y_train)
print("Training Accuracy:", best.score(X_train, y_train))
print("Testing Accuracy:", best.score(X_test, y_test))

Training Accuracy: 0.9257142857142857
Testing Accuracy: 0.64


### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [21]:
print(best.score(X_test, y_test))
print(best)

0.64
Pipeline(steps=[('scaler', StandardScaler()),
                ('DT', DecisionTreeClassifier(max_depth=5, random_state=0))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [20]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.5429167644854836
0.5427848101265823
Adewale Adeniji 0.542850787306033


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul> 
<li>The first step was to change the diffult train_size of the train_test_split to 0.7 so as to have more data for testing.
<li>Secondly, the max_depth of the DecisionTreeClassifier was set to 5 as this was found to produce the best result for the test data.
</ul>